## 日内分钟级数据的CVaR因子

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
tqdm.pandas()
# pd.set_option('display.max_rows', 50)

/mnt/datadisk2/aglv/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def cVaR_series(series, c_level=0.9, is_low=True):
    '''
    CVaR:分布的尾部积分, Parameters:
    c_level:p值,默认0.9
    is_low:返回的是否是左侧CVaR,=False时返回右侧,默认True
    '''
    right_VaR = series.quantile(c_level)
    left_VaR = series.quantile(1- c_level)

    if is_low:
        X_low = pd.Series(series[series < left_VaR])
        low_mean = X_low.mean() if len(X_low) > 0 else 0
        return low_mean
    else:
        X_high = pd.Series(series[series > right_VaR])
        high_mean = X_high.mean() if len(X_high) > 0 else 0
        return -high_mean

In [3]:
open2021 = pd.read_hdf('/mnt/datadisk2/aglv/aglv/lab_aglv/open_2021.h5')
open2021 = open2021.set_index(['Date', 'EndTime'])
open2021

InstrumentID      SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date     EndTime                                                               
20210104 09:31        9.64     14.10      8.95     12.45      2.86     75.70   
         09:32        9.64     14.01      8.82     12.35      2.86     76.46   
         09:33        9.60     13.97      8.84     12.36      2.84     76.17   
         09:34        9.60     14.00      8.83     12.37      2.85     76.10   
         09:35        9.58     13.90      8.83     12.33      2.85     75.74   
...                    ...       ...       ...       ...       ...       ...   
20211231 14:56        8.55     12.04      6.81     14.41      3.41     46.68   
         14:57        8.54     12.03      6.81     14.41      3.40     46.68   
         14:58        8.55     12.03      6.81     14.39      3.41     46.70   
         14:59        8.55     12.03      6.81     14.39      3.41     46.70   
         15:00        8.55     12.03      6.81     14.39      3.41     46.70   

InstrumentID      SH600010  SH600011  SH600012  SH600015  ...  SZ301190  \
Date     EndTime                                          ...             
20210104 09:31        1.17      4.50      6.14      6.22  ...       NaN   
         09:32        1.18      4.49      6.12      6.22  ...       NaN   
         09:33        1.17      4.47      6.16      6.20  ...       NaN   
         09:34        1.17      4.47      6.17      6.21  ...       NaN   
         09:35        1.17      4.46      6.16      6.21  ...       NaN   
...                    ...       ...       ...       ...  ...       ...   
20211231 14:56        2.78      9.64      7.07      5.59  ...     30.31   
         14:57        2.79      9.65      7.07      5.59  ...     30.28   
         14:58        2.79      9.65      7.06      5.60  ...     30.20   
         14:59        2.79      9.65      7.06      5.60  ...     30.20   
         15:00        2.79      9.65      7.06      5.60  ...     30.20   

InstrumentID      SH688210  SH688206  SH688265  SZ301166  SZ301189  SH688227  \
Date     EndTime                                                               
20210104 09:31         NaN       NaN       NaN       NaN       NaN       NaN   
         09:32         NaN       NaN       NaN       NaN       NaN       NaN   
         09:33         NaN       NaN       NaN       NaN       NaN       NaN   
         09:34         NaN       NaN       NaN       NaN       NaN       NaN   
         09:35         NaN       NaN       NaN       NaN       NaN       NaN   
...                    ...       ...       ...       ...       ...       ...   
20211231 14:56       41.48     36.95     69.92    104.04     60.49     32.14   
         14:57       41.51     36.93     70.00    103.70     60.32     32.12   
         14:58       41.48     36.73     70.05    103.20     60.05     32.09   
         14:59       41.48     36.73     70.05    103.30     60.02     32.09   
         15:00       41.48     36.73     70.05    103.30     60.02     32.09   

InstrumentID      SH688236  SZ301127  SH603176  
Date     EndTime                                
20210104 09:31         NaN       NaN       NaN  
         09:32         NaN       NaN       NaN  
         09:33         NaN       NaN       NaN  
         09:34         NaN       NaN       NaN  
         09:35         NaN       NaN       NaN  
...                    ...       ...       ...  
20211231 14:56       27.39     20.23      2.45  
         14:57       27.40     20.30      2.45  
         14:58       27.39     20.25      2.45  
         14:59       27.39     20.19      2.45  
         15:00       27.39     20.19      2.45  

[58320 rows x 4622 columns]

In [3]:
def min_data_cVaR(price:pd.DataFrame):
    def _min_data_cVaR(price_daily:pd.DataFrame):
        return_daily = price_daily.pct_change().fillna(0)
        
        return return_daily.apply(cVaR_series)
    
    return price.groupby(level='Date').progress_apply(_min_data_cVaR)

In [7]:
f1 = min_data_cVaR(open2021)
f1

InstrumentID,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301190,SH688210,SH688206,SH688265,SZ301166,SZ301189,SH688227,SH688236,SZ301127,SH603176
Date,,,,,,,,,,,,,,,,,,,,,
20210104,-0.002489,-0.002604,-0.005117,-0.002957,-0.006110,-0.003075,-0.008547,-0.003470,-0.002889,-0.002897,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210105,-0.001675,-0.002426,-0.006855,-0.001922,-0.004014,-0.002728,0.000000,-0.003102,-0.003773,-0.002218,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210106,-0.001678,-0.002765,-0.005206,-0.002439,-0.003835,-0.002591,0.000000,-0.002888,-0.004765,-0.001840,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210107,-0.002400,-0.003025,-0.006216,-0.003578,-0.003906,-0.002387,-0.008547,-0.003338,-0.003229,-0.002272,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20210108,-0.002419,-0.002861,-0.005919,-0.002562,-0.003690,-0.002498,0.000000,-0.003215,-0.004016,-0.002475,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211227,-0.001373,-0.002288,-0.001797,-0.002773,-0.004420,-0.001909,-0.005553,-0.007708,-0.002072,0.000000,...,-0.015806,-0.009663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20211228,-0.001472,-0.002883,-0.001591,-0.002156,-0.004972,-0.002129,-0.004163,-0.006616,-0.002195,0.000000,...,-0.005552,-0.008154,-0.016309,-0.010780,-0.019425,-0.019387,0.000000,0.000000,0.000000,0.0
20211229,-0.001870,-0.001966,-0.001919,-0.001751,-0.003259,-0.001980,-0.004474,-0.005607,-0.001945,-0.001789,...,-0.008270,-0.003561,-0.007330,-0.005403,-0.015898,-0.007706,0.000000,0.000000,0.000000,0.0


In [4]:
# f1.to_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor_test/fa_21_23/interday_cvar_neg_0.9_min.csv')

In [5]:
data2023 = pd.read_hdf('/mnt/datadisk2/aglv/aglv/lab_aglv/data2023.h5')
data2023

Open  Close   High    Low  LastVolume  \
Date     InstrumentID EndTime                                           
20230103 SZ000001     09:31    13.20  13.16  13.23  13.06   3725537.0   
                      09:32    13.16  13.14  13.18  13.11   1122500.0   
                      09:33    13.14  13.17  13.17  13.11   1284694.0   
                      09:34    13.17  13.20  13.20  13.16   1218457.0   
                      09:35    13.20  13.19  13.19  13.16   1066087.0   
...                              ...    ...    ...    ...         ...   
20231012 SH689009     14:56    33.94  33.88  33.94  33.88     23275.0   
                      14:57    33.92  33.89  33.92  33.86      9297.0   
                      14:58    33.89  33.89  33.89  33.89         0.0   
                      14:59    33.89  33.89  33.89  33.89         0.0   
                      15:00    33.77  33.77  33.77  33.77     62011.0   

                               LastTurnOver  
Date     InstrumentID EndTime                
20230103 SZ000001     09:31    4.894731e+07  
                      09:32    1.473833e+07  
                      09:33    1.688369e+07  
                      09:34    1.605823e+07  
                      09:35    1.404670e+07  
...                                     ...  
20231012 SH689009     14:56    7.893110e+05  
                      14:57    3.150930e+05  
                      14:58    0.000000e+00  
                      14:59    0.000000e+00  
                      15:00    2.094111e+06  

[222608001 rows x 6 columns]

In [7]:
open2023 = data2023['Open'].unstack(level='InstrumentID')
open2023

InstrumentID      SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date     EndTime                                                               
20230103 09:31        7.27     14.99      5.58     15.75      2.84     57.56   
         09:32        7.22     14.91      5.54     15.78      2.83     57.46   
         09:33        7.23     14.92      5.53     15.82      2.83     57.39   
         09:34        7.24     14.87      5.57     15.87      2.84     57.31   
         09:35        7.25     14.99      5.56     15.80      2.83     57.48   
...                    ...       ...       ...       ...       ...       ...   
20231012 14:56        7.13     10.87      5.79     19.03      2.82     37.33   
         14:57        7.13     10.87      5.78     18.98      2.81     37.33   
         14:58        7.13     10.87      5.79     19.04      2.82     37.33   
         14:59        7.13     10.87      5.79     19.04      2.82     37.33   
         15:00        7.14     10.88      5.79     18.98      2.81     37.33   

InstrumentID      SH600010  SH600011  SH600012  SH600015  ...  SZ301519  \
Date     EndTime                                          ...             
20230103 09:31        1.92      7.70      7.30      5.20  ...       NaN   
         09:32        1.92      7.61      7.32      5.18  ...       NaN   
         09:33        1.93      7.63      7.33      5.18  ...       NaN   
         09:34        1.94      7.67      7.33      5.18  ...       NaN   
         09:35        1.94      7.68      7.32      5.18  ...       NaN   
...                    ...       ...       ...       ...  ...       ...   
20231012 14:56        1.73      7.82     10.84      5.72  ...     23.04   
         14:57        1.72      7.80     10.84      5.71  ...     23.06   
         14:58        1.73      7.82     10.81      5.73  ...     23.06   
         14:59        1.73      7.82     10.81      5.73  ...     23.06   
         15:00        1.72      7.81     10.83      5.71  ...     23.05   

InstrumentID      SZ301520  SZ301525  SZ301528  SZ301529  SZ301533  SZ301548  \
Date     EndTime                                                               
20230103 09:31         NaN       NaN       NaN       NaN       NaN       NaN   
         09:32         NaN       NaN       NaN       NaN       NaN       NaN   
         09:33         NaN       NaN       NaN       NaN       NaN       NaN   
         09:34         NaN       NaN       NaN       NaN       NaN       NaN   
         09:35         NaN       NaN       NaN       NaN       NaN       NaN   
...                    ...       ...       ...       ...       ...       ...   
20231012 14:56       65.80    102.10     84.05     44.23     41.44     61.35   
         14:57       65.85    101.96     84.10     44.24     41.44     61.35   
         14:58       65.64    101.81     84.10     44.28     41.39     61.34   
         14:59       65.64    101.81     84.10     44.28     41.39     61.34   
         15:00       65.64    101.90     84.29     44.35     41.40     61.33   

InstrumentID      SZ301550  SZ301558  SZ301559  
Date     EndTime                                
20230103 09:31         NaN       NaN       NaN  
         09:32         NaN       NaN       NaN  
         09:33         NaN       NaN       NaN  
         09:34         NaN       NaN       NaN  
         09:35         NaN       NaN       NaN  
...                    ...       ...       ...  
20231012 14:56       50.62     14.10     22.73  
         14:57       50.38     14.08     22.72  
         14:58       50.18     14.00     22.69  
         14:59       50.18     14.00     22.69  
         15:00       50.30     14.01     22.69  

[44718 rows x 5111 columns]

In [8]:
cvar_neg = min_data_cVaR(open2023)
cvar_neg

InstrumentID,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301519,SZ301520,SZ301525,SZ301528,SZ301529,SZ301533,SZ301548,SZ301550,SZ301558,SZ301559
Date,,,,,,,,,,,,,,,,,,,,,
20230103,-0.002256,-0.002980,-0.002242,-0.002562,-0.003519,-0.002031,-0.005181,-0.003611,-0.001650,-0.002121,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20230104,-0.001712,-0.001748,-0.001859,-0.002272,-0.003460,-0.001781,0.000000,-0.003198,-0.001977,-0.002452,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20230105,-0.002010,-0.002385,-0.002014,-0.002472,-0.004489,-0.002062,0.000000,-0.003649,-0.001918,-0.002137,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20230106,-0.001772,-0.002407,-0.001874,-0.003111,-0.003474,-0.002270,0.000000,-0.003019,-0.001613,-0.003166,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20230109,-0.002511,-0.002597,-0.002282,-0.002461,-0.003688,-0.001811,-0.010152,-0.004065,-0.001886,-0.002235,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230928,-0.001666,-0.001611,-0.001812,-0.001356,0.000000,-0.001129,-0.005814,-0.002252,-0.002329,-0.002183,...,-0.001507,-0.013727,-0.003027,-0.001489,-0.002656,-0.002220,-0.002849,-0.006299,-0.017631,0.000000
20231009,-0.001954,-0.002545,-0.002074,-0.002048,-0.003546,-0.001756,-0.005814,-0.001997,-0.002192,-0.002453,...,-0.002317,-0.006879,-0.002496,-0.001717,-0.003965,-0.002224,-0.002927,-0.005438,-0.012245,0.000000
20231010,-0.002128,-0.001681,-0.002172,-0.001428,-0.003559,-0.001214,-0.006389,-0.002566,-0.001931,-0.002349,...,-0.001713,-0.005204,-0.003523,-0.002025,-0.003561,-0.002199,-0.002106,-0.010185,-0.008934,0.000000


In [9]:
eg = pd.read_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor/factor_829/interday_cvar_neg_0.9_21_22_min.csv')
eg

,Date,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,...,SZ301368,SZ301369,SZ301377,SZ301379,SZ301380,SZ301388,SZ301389,SZ301391,SZ301396,SZ301398
0,20210104,-0.002489,-0.002604,-0.005117,-0.002957,-0.006110,-0.003075,-0.008547,-0.003470,-0.002889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210105,-0.001675,-0.002426,-0.006855,-0.001922,-0.004014,-0.002728,0.000000,-0.003102,-0.003773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210106,-0.001678,-0.002765,-0.005206,-0.002439,-0.003835,-0.002591,0.000000,-0.002888,-0.004765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210107,-0.002400,-0.003025,-0.006216,-0.003578,-0.003906,-0.002387,-0.008547,-0.003338,-0.003229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210108,-0.002419,-0.002861,-0.005919,-0.002562,-0.003690,-0.002498,0.000000,-0.003215,-0.004016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,20221226,-0.001926,-0.003213,-0.002112,-0.001891,-0.003597,-0.002076,-0.005263,-0.002744,-0.002481,...,-0.001966,-0.001633,-0.001926,-0.001597,-0.002441,-0.002123,-0.001597,-0.004109,-0.002208,-0.003076
481,20221227,-0.001393,-0.004508,-0.002353,-0.002238,-0.003559,-0.003293,-0.005181,-0.002681,-0.002079,...,-0.002333,-0.002068,-0.002444,-0.001802,-0.002658,-0.002192,-0.002407,-0.003983,-0.002846,-0.004253
482,20221228,-0.001651,-0.003081,-0.004029,-0.002394,-0.007092,-0.002061,-0.005102,-0.004060,-0.001918,...,-0.002217,-0.001956,-0.001724,-0.001755,-0.003459,-0.001928,-0.001990,-0.003002,-0.002241,-0.003321
483,20221229,-0.002016,-0.003311,-0.002434,-0.003208,-0.003547,-0.002427,-0.007175,-0.003241,-0.002086,...,-0.001905,-0.002593,-0.002153,-0.001868,-0.002145,-0.002203,-0.001720,-0.002629,-0.002447,-0.002642


In [12]:
# eg.set_index('Date', inplace=True)
eg

,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301368,SZ301369,SZ301377,SZ301379,SZ301380,SZ301388,SZ301389,SZ301391,SZ301396,SZ301398
Date,,,,,,,,,,,,,,,,,,,,,
20210104,-0.002489,-0.002604,-0.005117,-0.002957,-0.006110,-0.003075,-0.008547,-0.003470,-0.002889,-0.002897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,-0.001675,-0.002426,-0.006855,-0.001922,-0.004014,-0.002728,0.000000,-0.003102,-0.003773,-0.002218,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,-0.001678,-0.002765,-0.005206,-0.002439,-0.003835,-0.002591,0.000000,-0.002888,-0.004765,-0.001840,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,-0.002400,-0.003025,-0.006216,-0.003578,-0.003906,-0.002387,-0.008547,-0.003338,-0.003229,-0.002272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,-0.002419,-0.002861,-0.005919,-0.002562,-0.003690,-0.002498,0.000000,-0.003215,-0.004016,-0.002475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221226,-0.001926,-0.003213,-0.002112,-0.001891,-0.003597,-0.002076,-0.005263,-0.002744,-0.002481,-0.001953,...,-0.001966,-0.001633,-0.001926,-0.001597,-0.002441,-0.002123,-0.001597,-0.004109,-0.002208,-0.003076
20221227,-0.001393,-0.004508,-0.002353,-0.002238,-0.003559,-0.003293,-0.005181,-0.002681,-0.002079,-0.002030,...,-0.002333,-0.002068,-0.002444,-0.001802,-0.002658,-0.002192,-0.002407,-0.003983,-0.002846,-0.004253
20221228,-0.001651,-0.003081,-0.004029,-0.002394,-0.007092,-0.002061,-0.005102,-0.004060,-0.001918,-0.002253,...,-0.002217,-0.001956,-0.001724,-0.001755,-0.003459,-0.001928,-0.001990,-0.003002,-0.002241,-0.003321


In [13]:
cvar = pd.concat([eg, cvar_neg])
cvar

,SH600000,SH600004,SH600006,SH600007,SH600008,SH600009,SH600010,SH600011,SH600012,SH600015,...,SZ301519,SZ301520,SZ301525,SZ301528,SZ301529,SZ301533,SZ301548,SZ301550,SZ301558,SZ301559
Date,,,,,,,,,,,,,,,,,,,,,
20210104,-0.002489,-0.002604,-0.005117,-0.002957,-0.006110,-0.003075,-0.008547,-0.003470,-0.002889,-0.002897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210105,-0.001675,-0.002426,-0.006855,-0.001922,-0.004014,-0.002728,0.000000,-0.003102,-0.003773,-0.002218,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210106,-0.001678,-0.002765,-0.005206,-0.002439,-0.003835,-0.002591,0.000000,-0.002888,-0.004765,-0.001840,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210107,-0.002400,-0.003025,-0.006216,-0.003578,-0.003906,-0.002387,-0.008547,-0.003338,-0.003229,-0.002272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210108,-0.002419,-0.002861,-0.005919,-0.002562,-0.003690,-0.002498,0.000000,-0.003215,-0.004016,-0.002475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230928,-0.001666,-0.001611,-0.001812,-0.001356,0.000000,-0.001129,-0.005814,-0.002252,-0.002329,-0.002183,...,-0.001507,-0.013727,-0.003027,-0.001489,-0.002656,-0.002220,-0.002849,-0.006299,-0.017631,0.000000
20231009,-0.001954,-0.002545,-0.002074,-0.002048,-0.003546,-0.001756,-0.005814,-0.001997,-0.002192,-0.002453,...,-0.002317,-0.006879,-0.002496,-0.001717,-0.003965,-0.002224,-0.002927,-0.005438,-0.012245,0.000000
20231010,-0.002128,-0.001681,-0.002172,-0.001428,-0.003559,-0.001214,-0.006389,-0.002566,-0.001931,-0.002349,...,-0.001713,-0.005204,-0.003523,-0.002025,-0.003561,-0.002199,-0.002106,-0.010185,-0.008934,0.000000


In [14]:
cvar.to_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor/factor_829/interday_cvar_neg_0.9_21_23_min.csv')